# KR260 AWGN DMA Test

### Plot function for use in this notebook
The first code block below defines a function that we will use for plotting data throughout this notebook. Note that the function has a `n_samples` argument so that we can limit the number of samples to plot. Plotting more than a few thousand samples can be very slow and consume a lot of RAM.

### Requirements
Install the following:

```bash
pip install ipympl
pip install ipywidgets
pip install jupyter_bokeh
```

### Hardware AWGN Block Implementation
##### Load Overlay

In [1]:
from pynq import Overlay
from pynq import allocate
from pynq import MMIO
import pynq.lib.dma
import time
import numpy as np
import math
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook, show

#import Python library from repo:
import sys
sys.path.append('/root/jupyter_notebooks/puch/FixedGain_tlast_debug')
import fpga.py.puch as puch
import fpga.lib.timestamp.sw.timestamp_regmap as timestamp_regmap
import fpga.lib.led_reg.sw.led_regmap as led_regmap
import fpga.lib.AWGN_GNG.sw.awgn_regmap as awgn_regmap


# Load the overlay
overlay = Overlay('../../overlays/KR260_AWGN_DMA_Stream/output/kr260_awgn_dma_stream.bit')

# Assign blocks to short vars
dma          = overlay.axi_dma_0
led_module   = overlay.led_reg_0
timestamp    = overlay.Timestamp_0
awgn         = overlay.gng_top_0

#### Display FPGA Timestamp

In [2]:
print("FPGA Build Timestamp:  " + puch.get_timestamp_str(timestamp))

FPGA Build Timestamp:  2025/4/5 9:16:54


#### Configure the AWGN

In [3]:
print("Noise Gain = " + str(awgn.read(awgn_regmap.RegMap.AWGN_NOISE_GAIN_ADDR)))
awgn.write(awgn_regmap.RegMap.AWGN_NOISE_GAIN_ADDR,4)
print("Noise Gain = " + str(awgn.read(awgn_regmap.RegMap.AWGN_NOISE_GAIN_ADDR)))
print("AWGN Enable = " + str(awgn.read(awgn_regmap.RegMap.AWGN_ENABLE_ADDR)))
awgn.write(awgn_regmap.RegMap.AWGN_ENABLE_ADDR,1)
print("AWGN Enable = " + str(awgn.read(awgn_regmap.RegMap.AWGN_ENABLE_ADDR)))

#awgn.write(gain_module.register_map.gain.address,4)
tBits, fBits = puch.get_format(awgn,awgn_regmap.RegMap.F_IN_ADDR)
print("F_IN : (" + str(tBits) + ", " + str(fBits) + ")")
tBits, fBits = puch.get_format(awgn,awgn_regmap.RegMap.F_OUT_ADDR)
print("F_OUT : (" + str(tBits) + ", " + str(fBits) + ")")
tBits, fBits = puch.get_format(awgn,awgn_regmap.RegMap.F_AWGN_ADDR)
print("F_AWGN : (" + str(tBits) + ", " + str(fBits) + ")")

type(fBits)

Noise Gain = 0
Noise Gain = 4
AWGN Enable = 0
AWGN Enable = 1
F_IN : (16, 14)
F_OUT : (16, 14)
F_AWGN : (16, 11)


int

### DMA Transfer Test

In [44]:
dma_block_size = 10000 #2**11;
iter_num = 2**4
rng = np.random.default_rng()
block_data = rng.integers(low=0, high=2^16, size=dma_block_size)
returned_data = rng.integers(low=0, high=2^16, size=dma_block_size)


# Allocate buffers for the input and output signals
in_buffer = allocate(shape=(dma_block_size,), dtype=np.int32)
out_buffer = allocate(shape=(dma_block_size,), dtype=np.int32)

num_blocks = 0
time_log = []
p = figure(title = "Block Transfer Rates", x_axis_label = 'Block Index', y_axis_label = 'Transfer Rate (sec)')
for i in range(iter_num):
    # Copy the samples to the in_buffer
    block_data = rng.integers(low=0, high=2^16, size=dma_block_size)
    
    start_time = time.time()
    np.copyto(in_buffer,block_data)
    
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    
    stop_time = time.time()
    np.copyto(returned_data,out_buffer)
    
    hw_exec_time = stop_time-start_time
    time_log.append(hw_exec_time)
    
    num_blocks += 1
        

print("Min Value                    : "+str(min(time_log)/2**3)+" ms")
print("Max Value                    : "+str(max(time_log)/2**3)+" ms")
print("Min Tranfer Rate             : "+str(32*dma_block_size/max(time_log)/1e6)+" Mbps")
print("Max Tranfer Rate             : "+str(32*dma_block_size/min(time_log)/1e6)+" Mbps")
print("Min Tranfer Rate             : "+str(32*dma_block_size/max(time_log)/1e6/32)+" MSPS")
print("Max Tranfer Rate             : "+str(32*dma_block_size/min(time_log)/1e6/32)+" MSPS")
print("Total Bit Transferred        : "+str((num_blocks)*dma_block_size*32))

output_notebook()
p = figure(title = "Block Transfer Rates", x_axis_label = 'Block Index', y_axis_label = 'Transfer Rate (sec)')
x=np.arange(0,len(time_log))
p.line(x,time_log,line_color="blue",line_width=2)
show(p)


# Free the buffers
in_buffer.close()
out_buffer.close()

Min Value                    : 5.906820297241211e-05 ms
Max Value                    : 0.00011673569679260254 ms
Min Tranfer Rate             : 342.65439877457237 Mbps
Max Tranfer Rate             : 677.1832896064581 Mbps
Min Tranfer Rate             : 10.707949961705387 MSPS
Max Tranfer Rate             : 21.161977800201814 MSPS
Total Bit Transferred        : 5120000


Loading BokehJS ...

### Read TVALID & TLAST Counters

In [45]:
awgn.write(awgn_regmap.RegMap.CNT_CTRL_ADDR,0x2)
tvalid_cnt = awgn.read(awgn_regmap.RegMap.TVALID_CNT_ADDR)
tlast_cnt = awgn.read(awgn_regmap.RegMap.TLAST_CNT_ADDR)
print("TVALID Count   : " + str(tvalid_cnt))
print("TLAST Count    : " + str(tlast_cnt))

TVALID Count   : 160000
TLAST Count    : 16


#### Clear Counters

In [42]:
awgn.write(awgn_regmap.RegMap.CNT_CTRL_ADDR,0x1)